# classification 
In questa parte andiamo a svolgere le task di classification del dataset.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict

In [4]:
X_train_dec = np.load("X_train_decimato.npy")
X_test_dec = np.load("X_test_decimato.npy")
print(X_train_dec.shape, X_test_dec.shape)

(1828, 38038) (624, 38038)


In [5]:
y_train_df = pd.read_csv('RavdessAudioOnlyRavdessAudioOnlyNumpy__Y_train.csv')
y_test_df = pd.read_csv('RavdessAudioOnlyRavdessAudioOnlyNumpy__Y_test.csv')

In [6]:
y_train = y_train_df[['vocal_channel']]
y_test = y_test_df[['vocal_channel']]

In [7]:
# can i try to cut the first seconds of the time series and latest seconds
X_train_cut = X_train_dec[:, 7500:15000]
X_test_cut = X_test_dec[:, 7500:15000]
print(X_train_cut.shape)
print(X_test_cut.shape)

(1828, 7500)
(624, 7500)


In [8]:
X_train = X_train_cut
X_test = X_test_cut

# time series classifier. 

In [9]:
from sklearn.neighbors import KNeighborsClassifier
#Main classification metrics and utilities
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
import kds #for the lift

#Classifiers
from sklearn.tree import DecisionTreeClassifier
from pyts.classification import KNeighborsClassifier
from tslearn.neighbors import KNeighborsTimeSeriesClassifier

In [10]:
clf = KNeighborsClassifier(n_neighbors=5, weights='uniform')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy 0.6410256410256411
F1-score [0.27272727 0.76170213]
              precision    recall  f1-score   support

        song       0.95      0.16      0.27       264
      speech       0.62      0.99      0.76       360

    accuracy                           0.64       624
   macro avg       0.79      0.58      0.52       624
weighted avg       0.76      0.64      0.55       624



adesso provo velocemente a vedere cosa cambia se normalizzo i dataset

In [11]:
# normalizzazione dei dati
from tslearn.preprocessing import TimeSeriesScalerMinMax

In [12]:
scaler = TimeSeriesScalerMinMax()
X_train_s = scaler.fit_transform(X_train).reshape(X_train.shape[0], X_train.shape[1])

In [13]:
scaler = TimeSeriesScalerMinMax()
X_test_s = scaler.fit_transform(X_test).reshape(X_test.shape[0], X_test.shape[1])

In [14]:
clf = KNeighborsClassifier(n_neighbors=5, weights='uniform')
clf.fit(X_train_s, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy 0.5769230769230769
F1-score [0.         0.73170732]
              precision    recall  f1-score   support

        song       0.00      0.00      0.00       264
      speech       0.58      1.00      0.73       360

    accuracy                           0.58       624
   macro avg       0.29      0.50      0.37       624
weighted avg       0.33      0.58      0.42       624



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


come è possibile notare i dati non vanno normalizzati. 

In [15]:
# CNN Classifier

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling1D
from keras.layers import Conv1D, Activation, Conv1D, BatchNormalization

2023-07-06 14:19:06.883001: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
def build_simple_cnn(n_timesteps, n_outputs):
    model = Sequential()
    
    model.add(Conv1D(filters=16, kernel_size=8, activation='relu', input_shape=(n_timesteps, 1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.3))
    
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.3))
    
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.3))
    
    model.add(GlobalAveragePooling1D())
    
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [18]:
X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(X_train_cnn, y_train, test_size=0.2, stratify=y_train)

n_timesteps, n_outputs, n_features = X_train_cnn.shape[1], len(np.unique(y_train_cnn)), 1 
print("TIMESTEPS: ", n_timesteps)
print("N. LABELS: ", n_outputs)

TIMESTEPS:  7500
N. LABELS:  2


In [19]:
cnn = build_simple_cnn(n_timesteps, n_outputs)

2023-07-06 14:24:41.222355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 7493, 16)          144       
                                                                 
 batch_normalization (BatchN  (None, 7493, 16)         64        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 7493, 16)          0         
                                                                 
 dropout (Dropout)           (None, 7493, 16)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 7489, 32)          2592      
                                                                 
 batch_normalization_1 (Batc  (None, 7489, 32)         128       
 hNormalization)                                        

In [21]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [22]:
rlr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)
mc = ModelCheckpoint('best_model_cnn.h5', monitor='val_loss', save_best_only=True)

callbacks = [rlr, mc]

batch_size = 16
mini_batch_size = int(min(X_train_cnn.shape[0]/10, batch_size))

In [26]:
X_train_cnn = np.array(X_train_cnn, dtype=np.float32)

y_train_cnn = np.array(y_train_cnn, dtype=np.float32)


ValueError: could not convert string to float: 'speech'

In [25]:
history_cnn = cnn.fit(X_train_cnn, y_train_cnn, epochs=5, batch_size=mini_batch_size, callbacks=callbacks,
                      validation_data=(X_val_cnn, y_val_cnn)).history

Epoch 1/5


2023-07-06 14:47:30.235999: W tensorflow/core/framework/op_kernel.cc:1807] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'Cast_43' defined at (most recent call last):
    File "/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/opt/anaconda3/lib/python3.9/asyncio/base_events.py", line 596, in run_forever
      self._run_once()
    File "/opt/anac